In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame()

In [3]:
count = 50000

while(True):
    cur = pd.read_csv("../../data/global acc/result" + str(count) + ".csv", skipinitialspace=True, header=None)
    df = df.append(cur, ignore_index=True)
    if count == 500000:
        break
    count += 50000
cur = pd.read_csv("../../data/global acc/result_final.csv", skipinitialspace=True, header=None)
df = df.append(cur, ignore_index=True)

In [4]:
len(df)

549503

In [5]:
df.columns = ['global_acc1','global_acc2','global_acc3']

In [6]:
t = pd.DataFrame.from_csv("../../data/linear_acceleration_grav_mag.csv")

In [7]:
t.columns

Index(['tester_id', 'TagName', 'Time', 'acc_val1', 'acc_val2', 'acc_val3',
       'acc_val4', 'gravity1', 'gravity2', 'gravity3', 'mag_val1', 'mag_val2',
       'mag_val3'],
      dtype='object')

In [8]:
b = t[['tester_id', 'TagName', 'Time']].copy()

In [9]:
for col in ['global_acc1','global_acc2','global_acc3']:
    b[col] = 0

b[['global_acc1','global_acc2','global_acc3']] = df[['global_acc1','global_acc2','global_acc3']].values
b['unix_timestamp'] = pd.to_datetime(b['Time']).values.astype("uint64") / 1000000000

In [10]:
b.to_csv("../../data/global_acc_df.csv")

In [16]:
from scipy.integrate import simps

In [ ]:
features = pd.DataFrame(columns=['unix_timestamp','TagName','tester_id',
                                 'v_1','v_2','v_3','d_1',
                                          'd_2','d_3'])
b_groups = b.groupby(['TagName','tester_id'])
count = 0
for key in b_groups.groups.keys():
    cur_group = b_groups.get_group(key)
    t = cur_group.unix_timestamp.values
    v_1 = []
    d_1 = []
    v_2 = []
    d_2 = []
    v_3 = []
    d_3 = []
    for i in range(len(t)):
        v_1.append(simps(0.8*cur_group.global_acc1.values[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        d_1.append(simps(v_1[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        v_2.append(simps(0.8*cur_group.global_acc2.values[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        d_2.append(simps(v_2[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        v_3.append(simps(0.8*cur_group.global_acc3.values[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        d_3.append(simps(v_3[:i+1], x=t[:i+1]))
    
    v_1 = np.array(v_1)
    d_1 = np.array(d_1)
    v_2 = np.array(v_2)
    d_2 = np.array(d_2)
    v_3 = np.array(v_3)
    d_3 = np.array(d_3)

    temp = np.concatenate((t.reshape(-1,1),np.array([key[0]]*len(t)).reshape(-1,1),np.array([key[1]]*len(t)).reshape(-1,1),d_1.reshape(-1,1),d_2.reshape(-1,1),d_3.reshape(-1,1),v_1.reshape(-1,1),v_2.reshape(-1,1),v_3.reshape(-1,1)), axis = 1)

    features = features.append(pd.DataFrame(temp, columns=['unix_timestamp','TagName','tester_id',
                                                           'v_1','v_2','v_3','d_1',
                                                           'd_2','d_3']), ignore_index=True)
    count+=1
    if (count%50==0):
        print(count)

    

In [ ]:
features.to_csv("../../data/global_acc_features_df.csv")